In [11]:
from google.cloud import bigquery
import os

try:
    # Authenticate with service account
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\SPOT\Documents\Ecochain\ecochain_secret.json"
    
    # Initialize client
    client = bigquery.Client(project="ecochain123")
    print(f"Connected to project: {client.project}")
    
    # Create dataset (supplychain) in US region
    dataset_id = f"{client.project}.supplychain"
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "US"
    
    dataset = client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset Created Successfully: {dataset.dataset_id} in {dataset.location}")
    
except Exception as e:
    print(f"Failed to Create Dataset: {e}")

Connected to project: ecochain123
Dataset Created Successfully: supplychain in US


In [13]:
%%bigquery --project {PROJECT_ID}

SELECT * FROM `ecochain123.supplychain.suppliers_with_images`
ORDER BY supplier_id DESC
LiMIT 15

Query is running:   0%|          |

Downloading:   0%|          |

,supplier_id,supplier_name,country,region,product_category,sub_category,total_eco_score,carbon_score,water_score,waste_score,...,partnership_status,annual_volume,cost_premium,risk_level,last_audit,audit_summary,image_url,recommendation,text_embedding,text_embedding_2
0,SUP5013,spotit,Ghana,Africa,Textiles,Organic Cotton,50.28761,58.68838,39.359247,64.351547,...,Active,100000,0.00,Low,2025-09-10,have keen interest on protecting the environment,https://storage.googleapis.com/ecochain-produc...,Neutral,"[0.007951375097036362, -0.00795552134513855, 0...",None
1,SUP5012,spotit,Ghana,Africa,Textiles,Organic Cotton,NaN,NaN,NaN,NaN,...,Active,100000,0.00,Low,2025-09-10,have keen interest on protecting the environment,https://storage.googleapis.com/ecochain-produc...,None,"[0.007951375097036362, -0.00795552134513855, 0...",None
2,SUP5011,spotit,Ghana,Africa,Textiles,Organic Cotton,NaN,NaN,NaN,NaN,...,Active,100000,0.00,Low,2025-09-10,has a keen interest in protecting,https://storage.googleapis.com/ecochain-produc...,None,"[-0.001667151926085353, 0.0009823257569223642,...",None
3,SUP5010,spotit,Ghana,Africa,Textiles,Organic Cotton,NaN,NaN,NaN,NaN,...,Active,100000,0.00,Low,2025-09-10,has a keen interest on protecting the environm...,https://storage.googleapis.com/ecochain-produc...,None,"[-0.005236229859292507, -0.007845994085073471,...",None
4,SUP5009,spotit,Ghana,Africa,Textiles,Organic Cotton,NaN,NaN,NaN,NaN,...,Active,100000,0.00,Low,2025-09-10,has keen interest in protecting the environment,https://storage.googleapis.com/ecochain-produc...,None,"[0.005196236073970795, -0.0032346125226467848,...",None
5,SUP5008,spotit,Ghana,Africa,Textiles,Organic Cotton,NaN,NaN,NaN,NaN,...,Active,100000,0.00,Low,2025-09-10,has keen interest in protecting the environment,https://storage.googleapis.com/ecochain-produc...,None,"[-0.007169068790972233, -0.008557850494980812,...",None
6,SUP5007,spotit,Ghana,Africa,Textiles,Organic Cotton,NaN,NaN,NaN,NaN,...,Active,100000,0.00,Low,2025-09-10,has keen interest in protecting the environment,https://storage.googleapis.com/ecochain-produc...,None,"[0.005196236073970795, -0.0032346125226467848,...",None
7,SUP5006,spotit,Ghana,Africa,Textiles,Organic Cotton,NaN,NaN,NaN,NaN,...,Active,100000,0.00,Low,2025-09-10,has keen interest in protecting the environment,https://storage.googleapis.com/ecochain-produc...,None,"[0.005196236073970795, -0.0032346125226467848,...",None
8,SUP5005,spotit,Ghana,Africa,Textiles,Organic Cotton,NaN,NaN,NaN,NaN,...,Active,100000,0.00,Low,2025-09-10,has keen interest in protecting the environment,https://storage.googleapis.com/ecochain-produc...,None,"[0.005196236073970795, -0.0032346125226467848,...",None
9,SUP5004,spotit,Ghana,Africa,Electronics,Organic Cotton,NaN,NaN,NaN,NaN,...,Active,100000,0.00,Low,2025-09-10,has keen interest in protecting the environment,https://storage.googleapis.com/ecochain-produc...,None,"[0.009442867711186409, -0.01360091008245945, 0...",None


In [ ]:
%%bigquery --project {PROJECT_ID}

DELETE FROM `ecochain123.supplychain.suppliers_with_images`
WHERE supplier_name = 'spotit'

Executing query with job ID: a81662a2-b7da-4138-a458-36b8cb1cb239
Query executing: 12.32s

OPEN YOUR GOOGLE CLOUD SDK TERMINAL AND RUN THIS TO CREATE A CONNECTION 

bq mk --connection --connection_type=CLOUD_RESOURCE --location=US test_connection

NOTE YOUR CONNECTION DETAILS

THEN RUN THIS IN YOUR CLOUD SDK TERMINAL TO ENABLE PERMISSION TO USE VERTEX MODELS

gcloud projects add-iam-policy-binding ecochain123 --member="serviceAccount:bqcx-141077011867-bnp6@gcp-sa-bigquery-condel.iam.gserviceaccount.com" --role="roles/aiplatform.user"

Loading the synthetic dataset

In [2]:
import pandas as pd
data = pd.read_csv("C:/Users/SPOT/Documents/Ecochain/ecochain_suppliers.csv")

In [5]:
# Constructing Full Table ID
table_ref = f"{dataset_id}.suppliers"

#setting up the load job config
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE)

job = client.load_table_from_dataframe(
    data, table_ref, job_config = job_config
)

job.result()

print(f"Loaded {job.output_rows} rows into {table_ref}.")

C:\Users\SPOT\Documents\churn\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Loaded 5000 rows into ecochain123.supplychain.suppliers.


Creating Models In Bigquery

In [2]:
try:
    query = """
        CREATE OR REPLACE MODEL `ecochain123.supplychain.gemini_model`
        REMOTE WITH CONNECTION `141077011867.us.test_connection` 
        OPTIONS (endpoint = 'gemini-2.5-flash'); 
    """
    job = client.query(query)
    job.result()  # wait for completion
    print('Model Created Successfully')

except Exception as e:
    print(f'An Error Occurred When Creating Model: {e}')


Model Created Successfully


In [57]:
try:
    query = """
        CREATE OR REPLACE MODEL `ecochain123.supplychain.embedding_model`
        REMOTE WITH CONNECTION `141077011867.us.test_connection` 
        OPTIONS (endpoint = 'gemini-embedding-001'); 
    """
    job = client.query(query)
    job.result()  # wait for completion
    print('Model Created Successfully')

except Exception as e:
    print(f'An Error Occurred When Creating Model: {e}')

Model Created Successfully


In [12]:
%load_ext google.cloud.bigquery

PROJECT_ID = "ecochain123"

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [4]:
%%bigquery --project {PROJECT_ID}

SELECT * FROM `ecochain123.supplychain.suppliers`
LIMIT 1

Query is running:   0%|          |

Downloading:   0%|          |

,supplier_id,supplier_name,country,region,product_category,sub_category,total_eco_score,carbon_score,water_score,waste_score,social_score,certification,partnership_status,annual_volume,cost_premium,risk_level,last_audit,audit_summary,text_embedding
0,SUP0221,"Eaton, Long and Smith",Argentina,Americas,Chemicals,Biodegradable Detergents,71.15,46.85,19.38,96.67,38.38,"ISO14001, Rainforest Alliance",Inactive,620485,7.25,Medium,2023-07-02,ISO14001 compliance confirmed; Rainforest Alli...,[]


Converting Embedding Columns For Similarity Search

In [21]:
%%bigquery --project {PROJECT_ID}

ALTER TABLE `ecochain123.supplychain.suppliers`
ADD COLUMN text_embedding ARRAY<FLOAT64>

Query is running:   0%|          |

""


In [7]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE TABLE `ecochain123.supplychain.suppliers_embeddings` AS
SELECT
  supplier_id,
  ml_generate_embedding_result AS text_embedding
FROM
  ML.GENERATE_EMBEDDING(
    MODEL `ecochain123.supplychain.embedding_model`,
    (
      SELECT
        supplier_id,
        CONCAT(
          "Category: ", product_category, "; ",
          "Sub-category: ", sub_category, "; ",
          "Certification: ", certification, "; ",
          "Audit: ", audit_summary
        ) AS content
      FROM `ecochain123.supplychain.suppliers`
    ),
    STRUCT(TRUE AS flatten_json_output)
  );

Query is running:   0%|          |

""


In [8]:
%%bigquery --project {PROJECT_ID}

UPDATE `ecochain123.supplychain.suppliers` t
SET t.text_embedding = s.text_embedding
FROM `ecochain123.supplychain.suppliers_embeddings` s
WHERE t.supplier_id = s.supplier_id;

Query is running:   0%|          |

""


In [9]:
%%bigquery --project {PROJECT_ID}

SELECT * FROM `ecochain123.supplychain.suppliers`
LIMIT 1

Query is running:   0%|          |

Downloading:   0%|          |

,supplier_id,supplier_name,country,region,product_category,sub_category,total_eco_score,carbon_score,water_score,waste_score,social_score,certification,partnership_status,annual_volume,cost_premium,risk_level,last_audit,audit_summary,text_embedding
0,SUP1896,"Jenkins, Ellison and Lowe",Argentina,Americas,Chemicals,Biodegradable Detergents,92.32,52.74,84.75,46.23,95.63,"Fair Trade, B-Corp, ISO14001",Under Review,220054,3.74,Medium,2021-03-28,ISO14001 compliance confirmed; Fair Trade prac...,"[0.02262956276535988, -0.014745780266821384, 0..."


Creating A GCS Bucket and adding prduct images to it, Due to the large amount of rows, images were grouped by subcategory

Ensure google-cloud-storage is installed 

In [ ]:
# Creating Bucket 
from google.cloud import storage

try:
    gcs_client = storage.Client()
    bucket_name = "ecochain-product-images"
    bucket = gcs_client.bucket(bucket_name)
    bucket.location = "US"
    bucket = gcs_client.create_bucket(bucket)
    
    # Upload images from local folder
    folder_path = "C:/Users/SPOT/Documents/Ecochain/Product images"
    for file_name in os.listdir(folder_path):
        blob = bucket.blob(file_name)
        blob.upload_from_filename(os.path.join(folder_path, file_name))
        blob.make_public()  # make individual file public
        print(f"Public URL: {blob.public_url}")

except Exception as e:
    print(f'An Error Occurred: {e}')

In [16]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE EXTERNAL TABLE `ecochain123.supplychain.object_images`
WITH CONNECTION `us.test_connection`
OPTIONS (
  object_metadata = 'SIMPLE',
  uris = ['gs://ecochain-product-images/*'] 
);

Query is running:   0%|          |

""


Renamed Columns to align with the sub category column in the suppliers dataset

In [50]:
import difflib

BUCKET_NAME = "ecochain-product-images"

# Get list of subcategories from BigQuery
query = """
SELECT DISTINCT sub_category
FROM `ecochain123.supplychain.suppliers`
"""
subcategories = [row.sub_category.lower() for row in client.query(query)]

# Get all files in bucket
bucket = gcs_client.bucket(BUCKET_NAME)
blobs = list(bucket.list_blobs())

filenames = [blob.name.lower() for blob in blobs]

for subcat in subcategories:
    expected_file = f"{subcat}.jpeg"

    if expected_file not in filenames:
        # Find close matches in existing files
        match = difflib.get_close_matches(expected_file, filenames, n=1, cutoff=0.7)
        if match:
            old_blob = bucket.blob(match[0])
            new_blob = bucket.blob(expected_file)

            # Copy and delete old file
            bucket.copy_blob(old_blob, bucket, new_blob.name)
            old_blob.delete()

            print(f"Renamed {match[0]} → {expected_file}")
        else:
            print(f"⚠️ No match found for {expected_file}")


Renamed biodegradable detergent.jpeg → biodegradable detergents.jpeg
Renamed coffe.jpeg → coffee.jpeg
Renamed green solvent.jpeg → green solvents.jpeg


In [17]:
%%bigquery --project {PROJECT_ID}

SELECT * 
FROM `ecochain123.supplychain.object_images`
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,uri,generation,content_type,size,md5_hash,updated,metadata,ref
0,gs://ecochain-product-images/biodegradable det...,1756736837757144,image/jpeg,5103,76e2c9daef15f35e90e3bdb941a3b34f,2025-09-01 14:27:25.161000+00:00,[],{'uri': 'gs://ecochain-product-images/biodegra...
1,gs://ecochain-product-images/cocoa.jpeg,1756736851255922,image/jpeg,11895,6c95b9922058337e008d6d92134f7e27,2025-09-01 14:27:37.581000+00:00,[],{'uri': 'gs://ecochain-product-images/cocoa.jp...
2,gs://ecochain-product-images/coffe.jpeg,1756736864652577,image/jpeg,14107,85822d59d95c6492bb06801470153b9d,2025-09-01 14:27:46.601000+00:00,[],{'uri': 'gs://ecochain-product-images/coffe.jp...
3,gs://ecochain-product-images/compostable paper...,1756736869649316,image/jpeg,8757,8afd92bbe2b6b5950b454d7271eeab12,2025-09-01 14:27:51.540000+00:00,[],{'uri': 'gs://ecochain-product-images/composta...
4,gs://ecochain-product-images/eco paints.jpeg,1756736876051478,image/jpeg,6654,5d6bf7775a734e5a5499108013cefd1e,2025-09-01 14:27:58.087000+00:00,[],{'uri': 'gs://ecochain-product-images/eco pain...


In [51]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE TABLE `ecochain123.supplychain.suppliers_with_images` AS
SELECT
  s.*,
  o.uri AS image_url
FROM
  `ecochain123.supplychain.suppliers` AS s
LEFT JOIN
  `ecochain123.supplychain.object_images` AS o
ON
  -- Normalize sub_category
  REGEXP_REPLACE(
    REGEXP_REPLACE(LOWER(TRIM(s.sub_category)), r'[^a-z0-9 ]', ''),  -- remove punctuation
    r' +', ' '  -- collapse multiple spaces
  )
  =
  -- Normalize filename extracted from URI
  REGEXP_REPLACE(
    REGEXP_REPLACE(
      LOWER(TRIM(REGEXP_EXTRACT(o.uri, r'.*/([^/]+)\.(?:jpeg|jpg|png)$'))),  -- only 1 capturing group now
      r'[^a-z0-9 ]', ''  -- remove punctuation
    ),
    r' +', ' '  -- collapse multiple spaces
  )

Query is running:   0%|          |

""


WE HAVE SUCCESSFULLY CREATED THE IMAGE URLS AND ADDED IT TO THE DATASET 

In [53]:
try:
    query = '''
        SELECT * FROM `ecochain123.supplychain.suppliers_with_images`
        '''
    df = client.query(query).to_dataframe()

except Except as e:
    print( f'AN ERROR OCCURRED: {e}')

In [55]:
df.isnull().sum()

supplier_id           0
supplier_name         0
country               0
region                0
product_category      0
sub_category          0
total_eco_score       0
carbon_score          0
water_score           0
waste_score           0
social_score          0
certification         0
partnership_status    0
annual_volume         0
cost_premium          0
risk_level            0
last_audit            0
audit_summary         0
text_embedding        0
image_url             0
dtype: int64

Now To add the recommendation column using AI.GENERATE_TABLE

In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE TABLE `ecochain123.supplychain.recommendation` AS
SELECT
  supplier_id,
  total_eco_score,
  Recommendation
FROM
  AI.GENERATE_TABLE(
    MODEL `ecochain123.supplychain.gemini_model`,
    (
      SELECT
        supplier_id,
        total_eco_score,
        CONCAT(
          "Using the following keywords (Preferred, Neutral, or Avoid) categorise this score. ",
          "If above 80, it is Preferred; ",
          "If between 50 and 80, it is Neutral; ",
          "If below 50, it is Avoid. ",
          "Score: ", CAST(total_eco_score AS STRING)
        ) AS prompt
      FROM 
        `ecochain123.supplychain.suppliers_with_images`
    ),
    STRUCT("Recommendation STRING" AS output_schema)
  );


Query is running:   0%|          |

In [8]:
%%bigquery --project {PROJECT_ID}

ALTER TABLE `ecochain123.supplychain.suppliers_with_images`
ADD COLUMN Recommendation STRING;

Query is running:   0%|          |

""


In [9]:
%%bigquery --project {PROJECT_ID}

UPDATE `ecochain123.supplychain.suppliers_with_images` t
SET t.Recommendation = s.Recommendation
FROM `ecochain123.supplychain.recommendation` s
WHERE t.supplier_id = s.supplier_id;

Query is running:   0%|          |

""
